In [1]:
%matplotlib inline

In [3]:
import pandas as pd
from Bio import SeqIO, Entrez
Entrez.email = 'adam.hockenberry@utexas.edu'
import time
import glob

# This database might be great to use, actually. But it only became available in this easy to use form after NCBI re-did a few things and this project was largely complete.

Worth considering (one day) diving deeper into what is available here. 

# First make the starting datatable better

In [120]:
# df = pd.read_csv('../../Data/Other_possible_dbs/NCBI_phage_db/all_complete_phage_info.csv')
df = pd.read_csv('../../Data/Other_possible_dbs/NCBI_phage_db/all_complete_phage_info_11_2020.csv')
print(df.shape)
df.head()

(19336, 20)


,Accession,SRA_Accession,Release_Date,Species,Genus,Family,Length,Sequence_Type,Nuc_Completeness,Genotype,Segment,Authors,Publications,Geo_Location,USA,Host,Isolation_Source,Collection_Date,BioSample,GenBank_Title
0,NC_050152,NaN,2020-08-13T00:00:00Z,Escherichia virus P1,Punavirus,Myoviridae,101660,RefSeq,complete,NaN,NaN,"Lobocka,M.B., Rusin,M., Bugajska,O., Dobruk,A....",NaN,NaN,NaN,Enterobacteriaceae,NaN,NaN,NaN,"Enterobacteria phage P7, complete genome"
1,NC_050153,NaN,2020-08-13T00:00:00Z,Escherichia phage vB_EcoM-Ro157c2YLVW,Punavirus,Myoviridae,80732,RefSeq,complete,NaN,NaN,"Liao,Y.-T., Quintela,I.A., Bridges,D.F., Liu,F...",NaN,USA,NaN,Escherichia,NaN,2017-05-03,NaN,"Escherichia phage vB_EcoM-Ro157c2YLVW, complet..."
2,NC_050154,NaN,2020-08-13T00:00:00Z,Escherichia phage D6,NaN,Myoviridae,91159,RefSeq,complete,NaN,NaN,"Gilcrease,E.B., Casjens,S.R.",NaN,NaN,NaN,Escherichia coli,NaN,NaN,NaN,"Escherichia phage D6, complete genome"
3,NC_050143,NaN,2020-08-10T00:00:00Z,Pseudomonas phage datas,Pbunavirus,Myoviridae,60746,RefSeq,complete,NaN,NaN,"Olsen,N.S., Forero-Junco,L., Kot,W., Hansen,L.H.",NaN,Denmark,NaN,Pseudomonas aeruginosa PAO1,NaN,2017,NaN,"Pseudomonas phage datas, complete genome"
4,NC_050144,NaN,2020-08-10T00:00:00Z,Pseudomonas phage Epa14,Pbunavirus,Myoviridae,65797,RefSeq,complete,NaN,NaN,"Jacobs,A.C., Freyberger,H.R., Farlow,J., Watte...",NaN,NaN,NaN,Pseudomonas sp.,NaN,NaN,NaN,"Pseudomonas phage Epa14, complete genome"


In [121]:
df['Family'].value_counts()

Siphoviridae          7308
Myoviridae            2972
Microviridae          2839
Autographiviridae     1148
Podoviridae           1120
Herelleviridae         394
Drexlerviridae         321
Demerecviridae         251
Inoviridae             184
Ackermannviridae       167
Cystoviridae            46
Leviviridae             39
Chaseviridae            38
Tectiviridae            26
Corticoviridae           6
Sphaerolipoviridae       4
Plasmaviridae            2
Name: Family, dtype: int64

In [8]:
df = df[df['Host'].isnull()==False]
print(df.shape)

(14361, 20)


In [9]:
df['Host'].value_counts()

Mycolicibacterium smegmatis MC2 155                       1788
Escherichia coli                                           906
Gordonia terrae                                            457
Lactococcus lactis                                         401
Klebsiella pneumoniae                                      368
                                                          ... 
Aerococcus viridans                                          1
Salmonella enterica subsp. enterica serovar Braenderup       1
Pseudoalteromonas sp. YD6                                    1
Salmonella enterica subsp. enterica serovar Saintpaul        1
Schaalia odontolytica                                        1
Name: Host, Length: 1019, dtype: int64

**Better treatment of host taxonomy**

In [23]:
taxonomies_to_fetch = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
problematic_hosts = []
for host in list(set(df['Host']))[:]:
    matching_indices = df[df['Host']==host].index
    if df.loc[matching_indices]['Host_superkingdom_id'].isnull().all()==False:
        print('Skipping')
        continue
    print(host)
    handle = Entrez.esearch(db='Taxonomy', term=host)
    record = Entrez.read(handle)
    if len(record['IdList']) != 1:
        problematic_hosts.append(host)
        continue
    record_id = record['IdList'][0]
    handle = Entrez.efetch(db="Taxonomy", id=record_id, retmode="xml")
    records = Entrez.read(handle)
    if len(records) != 1:
        problematic_hosts.append(host)
        continue        
    record = records[0]
    ###First get the genetic code right
    df.at[matching_indices, 'GeneticCode'] = record['GeneticCode']['GCId']
    ###Now possibly add the record itself
    if record ['Rank'] in taxonomies_to_fetch:
        df.at[matching_indices, 'Host_{}_id'.format(record['Rank'])] = record['TaxId']
        df.at[matching_indices, 'Host_{}_name'.format(record['Rank'])] = record['ScientificName']
    ###Finally go through the lineage
    lineage = record['LineageEx']
    for i in lineage:
        if i['Rank'] in taxonomies_to_fetch:
            df.at[matching_indices, 'Host_{}_id'.format(i['Rank'])] = i['TaxId']
            df.at[matching_indices, 'Host_{}_name'.format(i['Rank'])] = i['ScientificName']
    time.sleep(10)

Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Escherichia coli O157:H43 str. T22
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Escherichia coli O145:NM
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Nodularia
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Gordonia
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Esc

Lysinibacillus sphaericus C3-41
Thermus sp. TC4
Lactococcus garvieae Lg2
Pseudomonas syringae
Morganella morganii
Microcystis aeruginosa
Enterococcus faecalis 62
Synechococcus sp. WH 8018
Fusobacterium nucleatum
Streptococcus intermedius
Vibrio cholerae O1
Streptococcus anginosus
Sphaerotilus natans
Idiomarinaceae bacterium N2-2
Vibrio metschnikovii
Streptococcus pneumoniae BS458
Acaryochloris marina
Rhizobium gallicum
Acinetobacter baumannii A118
Streptomyces toxytricini
Sodalis glossinidius
Mycolicibacterium aichiense
Escherichia coli C
Streptococcus pneumoniae ST556
Lactococcus lactis subsp. cremoris TIFN7
Streptococcus pneumoniae GA47033
Bacteroides intestinalis
Polaribacter
Streptomyces sp.
Lactobacillus
Epibacterium mobile
Microbacterium natoriense
Caulobacter vibrioides CB15
Streptococcus equinus
Trichormus variabilis
Plesiomonas sp.
Streptococcus parauberis KCTC 11980BP
Streptococcus suis ST1
Aeromonas rivipollensis
Pseudomonas aeruginosa CHA
Myxococcus xanthus
Sphingomonas pau

In [34]:
df['Host_species_name'].value_counts()

Mycolicibacterium smegmatis    1978
Escherichia coli               1187
Salmonella enterica             423
Pseudomonas aeruginosa          400
Gordonia terrae                 382
                               ... 
Salipiger profundus               1
Pseudomonas sp. 1-1-1b            1
Rothia dentocariosa               1
Phaeobacter inhibens              1
Microcystis elabens               1
Name: Host_species_name, Length: 549, dtype: int64

In [35]:
df['GeneticCode'].value_counts()

11    12021
1       270
4        15
Name: GeneticCode, dtype: int64

**Fix the problematic hosts**

In [29]:
print(len(set(problematic_hosts)))
problematic_hosts

27


['Escherichia coli O157:H43 str. T22',
 'Escherichia coli O145:NM',
 'Nodularia',
 'Gordonia',
 'Escherichia coli O104:H4',
 'Escherichia coli BL21(DE3)',
 'Bacillus',
 'Escherichia coli O157:H7 str. EDL933',
 '[Bacillus] clarkii',
 'Yersinia enterocolitica (type O:3)',
 'Streptomyces coelicolor A3(2)',
 'Edwardsiella',
 '[Brevibacterium] flavum',
 'Curvibacter sp. AEP1.3',
 'Streptococcus anginosus MAS624',
 'Morganella sp. (in: Bacteria)',
 'Clostridium saccharoperbutylacetonicum N1-4(HMT)',
 'Methylophilaceae bacterium IMCC19250',
 'Erwinia horticola',
 'Escherichia coli O157:H7',
 'Escherichia coli O111:H2',
 'Escherichia coli O103:H25 str. NIPH-11060424',
 'Escherichia coli DH5[alpha]',
 'Bacillus sp. (in: Bacteria)',
 'Yersinia',
 'Phormidium sp. MIS-PhA',
 'Paracoccus']

**Cases with special characters**

In [56]:
still_problematic = []
for host in problematic_hosts:
    matching_indices = df[df['Host']==host].index
    host = host.replace(':', ' ')
    host = host.replace('[', '').replace(']', '')
    host = host.replace('(', ' ').replace(')', ' ')
    if df.loc[matching_indices]['Host_superkingdom_id'].isnull().all()==False:
        print('Skipping')
        continue
    print(host)
    handle = Entrez.esearch(db='Taxonomy', term=host)
    record = Entrez.read(handle)
    if len(record['IdList']) != 1:
        still_problematic.append(host)
        continue
    record_id = record['IdList'][0]
    handle = Entrez.efetch(db="Taxonomy", id=record_id, retmode="xml")
    records = Entrez.read(handle)
    if len(records) != 1:
        still_problematic.append(host)
        continue        
    record = records[0]
    ###First get the genetic code right
    df.at[matching_indices, 'GeneticCode'] = record['GeneticCode']['GCId']
    ###Now possibly add the record itself
    if record ['Rank'] in taxonomies_to_fetch:
        df.at[matching_indices, 'Host_{}_id'.format(record['Rank'])] = record['TaxId']
        df.at[matching_indices, 'Host_{}_name'.format(record['Rank'])] = record['ScientificName']
    ###Finally go through the lineage
    lineage = record['LineageEx']
    for i in lineage:
        if i['Rank'] in taxonomies_to_fetch:
            df.at[matching_indices, 'Host_{}_id'.format(i['Rank'])] = i['TaxId']
            df.at[matching_indices, 'Host_{}_name'.format(i['Rank'])] = i['ScientificName']
    time.sleep(10)

Escherichia coli O157 H43 str. T22
Escherichia coli O145 NM
Nodularia
Gordonia
Escherichia coli O104 H4
Escherichia coli BL21 DE3 
Bacillus
Escherichia coli O157 H7 str. EDL933
Bacillus clarkii
Yersinia enterocolitica  type O 3 
Streptomyces coelicolor A3 2 
Edwardsiella
Brevibacterium flavum
Curvibacter sp. AEP1.3
Streptococcus anginosus MAS624
Morganella sp.  in  Bacteria 
Clostridium saccharoperbutylacetonicum N1-4 HMT 
Methylophilaceae bacterium IMCC19250
Erwinia horticola
Escherichia coli O157 H7
Escherichia coli O111 H2
Escherichia coli O103 H25 str. NIPH-11060424
Escherichia coli DH5alpha
Bacillus sp.  in  Bacteria 
Yersinia
Phormidium sp. MIS-PhA
Paracoccus


In [57]:
print(len(set(still_problematic)))
still_problematic

11


['Nodularia',
 'Gordonia',
 'Bacillus',
 'Edwardsiella',
 'Curvibacter sp. AEP1.3',
 'Streptococcus anginosus MAS624',
 'Methylophilaceae bacterium IMCC19250',
 'Erwinia horticola',
 'Yersinia',
 'Phormidium sp. MIS-PhA',
 'Paracoccus']

**Cases with annoying genuses**

In [60]:
really_problematic = []
for host in still_problematic:
    matching_indices = df[df['Host']==host].index
    host = host.replace(':', ' ')
    host = host.replace('[', '').replace(']', '')
    host = host.replace('(', ' ').replace(')', ' ')
    if df.loc[matching_indices]['Host_superkingdom_id'].isnull().all()==False:
        print('Skipping')
        continue
    print(host)
    handle = Entrez.esearch(db='Taxonomy', term=host)
    record = Entrez.read(handle)
    if len(record['IdList']) == 0:
        really_problematic.append(host)
        continue
    temp_ids = record['IdList']
    successful_ids = []
    for record_id in temp_ids:
        handle = Entrez.efetch(db="Taxonomy", id=record_id, retmode="xml")
        records = Entrez.read(handle)
        if len(records) != 1:
            really_problematic.append(host)
            continue        
        record = records[0]
        lineage = record['LineageEx']
        for i in lineage:
            if i['Rank']=='superkingdom' and i['ScientificName'] =='Bacteria':
                successful_ids.append(record_id)
    if len(successful_ids) == 1:
        record_id = successful_ids[0]
    else:
        really_problematic.append(host)
    
    handle = Entrez.efetch(db="Taxonomy", id=record_id, retmode="xml")
    records = Entrez.read(handle)
    if len(records) != 1:
        really_problematic.append(host)
        continue        
    record = records[0]
    ###First get the genetic code right
    df.at[matching_indices, 'GeneticCode'] = record['GeneticCode']['GCId']
    ###Now possibly add the record itself
    if record ['Rank'] in taxonomies_to_fetch:
        df.at[matching_indices, 'Host_{}_id'.format(record['Rank'])] = record['TaxId']
        df.at[matching_indices, 'Host_{}_name'.format(record['Rank'])] = record['ScientificName']
    ###Finally go through the lineage
    lineage = record['LineageEx']
    for i in lineage:
        if i['Rank'] in taxonomies_to_fetch:
            df.at[matching_indices, 'Host_{}_id'.format(i['Rank'])] = i['TaxId']
            df.at[matching_indices, 'Host_{}_name'.format(i['Rank'])] = i['ScientificName']
    time.sleep(5)

Nodularia
Gordonia
Bacillus
Edwardsiella
Curvibacter sp. AEP1.3
Streptococcus anginosus MAS624
Methylophilaceae bacterium IMCC19250
Erwinia horticola
Yersinia
Phormidium sp. MIS-PhA
Paracoccus


In [61]:
print(len(really_problematic))
print(really_problematic)

5
['Curvibacter sp. AEP1.3', 'Streptococcus anginosus MAS624', 'Methylophilaceae bacterium IMCC19250', 'Erwinia horticola', 'Phormidium sp. MIS-PhA']


In [72]:
answers = [2685271, 1328, 2030816, 551, 1199]

In [75]:
taxonomies_to_fetch = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
problematic_hosts = []
for host, answer in zip(really_problematic, answers):
    matching_indices = df[df['Host']==host].index
    if df.loc[matching_indices]['Host_superkingdom_id'].isnull().all()==False:
        print('Skipping')
        continue
    print(host)
    record_id = str(answer)
    handle = Entrez.efetch(db="Taxonomy", id=record_id, retmode="xml")
    records = Entrez.read(handle)    
    record = records[0]
    ###First get the genetic code right
    df.at[matching_indices, 'GeneticCode'] = record['GeneticCode']['GCId']
    ###Now possibly add the record itself
    if record ['Rank'] in taxonomies_to_fetch:
        df.at[matching_indices, 'Host_{}_id'.format(record['Rank'])] = record['TaxId']
        df.at[matching_indices, 'Host_{}_name'.format(record['Rank'])] = record['ScientificName']
    ###Finally go through the lineage
    lineage = record['LineageEx']
    for i in lineage:
        if i['Rank'] in taxonomies_to_fetch:
            df.at[matching_indices, 'Host_{}_id'.format(i['Rank'])] = i['TaxId']
            df.at[matching_indices, 'Host_{}_name'.format(i['Rank'])] = i['ScientificName']
    time.sleep(10)

Skipping
Skipping
Skipping
Skipping
Skipping


**Write the new and improved file**

In [82]:
df['Host_superkingdom_name'].value_counts()

Bacteria     12211
Eukaryota      270
Archaea          2
Name: Host_superkingdom_name, dtype: int64

In [83]:
df = df[df['Host_superkingdom_name']=='Bacteria']
print(df.shape)

(12211, 33)


In [89]:
df.to_csv('../Data/NCBI_phage_db/all_complete_phage_info_HOSTTAXONOMY_05152020.tsv', sep='\t', index=False)

# Select the rows that I care about

In [122]:
df = pd.read_csv('../../Data/Other_possible_dbs/NCBI_phage_db/all_complete_phage_info_HOSTTAXONOMY_05152020.tsv', sep='\t')
print(df.shape)
df.head()

(12211, 33)


,Accession,Release_Date,Species,Genus,Family,Length,Sequence_Type,Nuc_Completeness,Genotype,Segment,...,Host_phylum_id,Host_phylum_name,Host_class_id,Host_class_name,Host_order_id,Host_order_name,Host_family_id,Host_family_name,Host_species_id,Host_species_name
0,NC_045425,2019-12-16T00:00:00Z,Thermus virus OH3,NaN,Inoviridae,5688,RefSeq,complete,NaN,NaN,...,1297.0,Deinococcus-Thermus,188787.0,Deinococci,68933.0,Thermales,188786.0,Thermaceae,274.0,Thermus thermophilus
1,NC_043767,2019-07-20T00:00:00Z,Mycobacterium virus TA17a,Rosebushvirus,Siphoviridae,67324,RefSeq,complete,NaN,NaN,...,201174.0,Actinobacteria,1760.0,Actinobacteria,85007.0,Corynebacteriales,1762.0,Mycobacteriaceae,1772.0,Mycolicibacterium smegmatis
2,NC_043027,2019-06-28T00:00:00Z,Bacillus virus PBS1,NaN,Myoviridae,252197,RefSeq,complete,NaN,NaN,...,1239.0,Firmicutes,91061.0,Bacilli,1385.0,Bacillales,186817.0,Bacillaceae,1423.0,Bacillus subtilis
3,NC_043028,2019-06-28T00:00:00Z,Xanthomonas virus Xf109,NaN,Inoviridae,7190,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,135614.0,Xanthomonadales,32033.0,Xanthomonadaceae,347.0,Xanthomonas oryzae
4,NC_043029,2019-06-28T00:00:00Z,Stenotrophomonas virus SMA6,NaN,Inoviridae,7648,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,135614.0,Xanthomonadales,32033.0,Xanthomonadaceae,40324.0,Stenotrophomonas maltophilia


In [123]:
df = df[df['Host_species_id'].isnull()==False]
print(df.shape)
df['Host_species_id'] = df['Host_species_id'].astype(int)
print(df.shape)
df = df[df['GeneticCode']==11]
print(df.shape)
df.head()

(11150, 33)
(11150, 33)
(11141, 33)


,Accession,Release_Date,Species,Genus,Family,Length,Sequence_Type,Nuc_Completeness,Genotype,Segment,...,Host_phylum_id,Host_phylum_name,Host_class_id,Host_class_name,Host_order_id,Host_order_name,Host_family_id,Host_family_name,Host_species_id,Host_species_name
0,NC_045425,2019-12-16T00:00:00Z,Thermus virus OH3,NaN,Inoviridae,5688,RefSeq,complete,NaN,NaN,...,1297.0,Deinococcus-Thermus,188787.0,Deinococci,68933.0,Thermales,188786.0,Thermaceae,274,Thermus thermophilus
1,NC_043767,2019-07-20T00:00:00Z,Mycobacterium virus TA17a,Rosebushvirus,Siphoviridae,67324,RefSeq,complete,NaN,NaN,...,201174.0,Actinobacteria,1760.0,Actinobacteria,85007.0,Corynebacteriales,1762.0,Mycobacteriaceae,1772,Mycolicibacterium smegmatis
2,NC_043027,2019-06-28T00:00:00Z,Bacillus virus PBS1,NaN,Myoviridae,252197,RefSeq,complete,NaN,NaN,...,1239.0,Firmicutes,91061.0,Bacilli,1385.0,Bacillales,186817.0,Bacillaceae,1423,Bacillus subtilis
3,NC_043028,2019-06-28T00:00:00Z,Xanthomonas virus Xf109,NaN,Inoviridae,7190,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,135614.0,Xanthomonadales,32033.0,Xanthomonadaceae,347,Xanthomonas oryzae
4,NC_043029,2019-06-28T00:00:00Z,Stenotrophomonas virus SMA6,NaN,Inoviridae,7648,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,135614.0,Xanthomonadales,32033.0,Xanthomonadaceae,40324,Stenotrophomonas maltophilia


In [124]:
df['Host_species_id'].value_counts()[:20]

1772       1978
562        1279
28901       423
287         400
2055        382
1358        358
573         251
1771959     231
1280        221
104336      208
1308        199
1747        182
1313        141
470         116
666         115
1351        110
1428        105
1314        101
1911         97
599          79
Name: Host_species_id, dtype: int64

In [73]:
for host_taxid in [562]:
    temp_df = df[df['Host_species_id']==host_taxid]
    print(temp_df.shape)
    assert len(set(list(temp_df['Accession']))) == len(list(temp_df['Accession']))
    selected_accession_numbers = list(temp_df['Accession'])
    all_genomes = SeqIO.parse('../../Data/Other_possible_dbs/NCBI_phage_db/all_complete_phage_nts.fasta', 'fasta')
    for genome in all_genomes:
        if genome.id in selected_accession_numbers:
            with open('../../Data/Other_possible_dbs/NCBI_phage_db/phage_genomes/{}.fasta'.format(genome.id), 'w') as outfile:
                SeqIO.write(genome, outfile, 'fasta')

(1279, 33)


# Split apart the combined genome `fasta` file to write the genomes that I selected

In [14]:
genome = SeqIO.read('../../Data/Other_possible_dbs/NCBI_phage_db/phage_genomes/NC_001416.fasta', 'fasta')
# for genome_file in glob.glob('../Data/NCBI_phage_db/phage_genomes/*.fasta'):
#     genome = SeqIO.read(genome_file, 'fasta')

In [16]:
hits = []
all_cds = SeqIO.parse('../../Data/Other_possible_dbs/NCBI_phage_db/all_complete_phage_CDS.fasta', 'fasta')
for cds in all_cds:
    if genome.id in cds.id.split(':')[0]:
        hits.append(cds)
print(len(hits))

73


In [17]:
print(len(hits))

73


In [23]:
for hit in hits:
    reverse_complement = False
    if 'join' in hit.id:
        continue
    if 'complement(' == hit.id[:11]:
        print('here')
        reverse_complement = True
#         hit.id = hit.id[11:-1]
    if len(hit.id.split(':')) == 2:
        region = hit.id.split(':')[-1]
        if len(region.split('..')) == 2:
            start = int(region.split('..')[0])
            stop = int(region.split('..')[-1][:-1])
    print(start, stop)
    print(genome.seq[start-1:stop]==hit.seq)

45192 4550
False
46186 4636
False
here
29118 29285
True
34482 3503
False
here
46459 46752
True
here
47042 47575
True
47738 4794
False
here
20147 20767
True
here
22686 23918
True
here
30839 31024
True
here
31005 31196
True
here
31169 31351
True
here
34271 34357
True
here
35037 35438
True
191 73
False
711 263
False
2633 283
False
2836 443
False
4418 573
False
5132 573
False
5747 607
False
6135 716
False
7202 760
False
7612 796
False
7977 855
False
8552 894
False
8955 969
False
9711 1013
False
10115 1054
False
10542 1310
False
13100 1342
False
13429 1412
False
14276 1487
False
14773 1544
False
15505 1890
False
18965 1958
False
19650 2085
False
21029 2197
False
21973 2255
False
here
24509 25399
True
here
25396 26973
True
here
27812 28882
True
here
28860 29078
True
here
29374 29655
True
here
29847 30395
True
here
31348 32028
True
here
32025 32810
True
here
32816 33232
True
here
33187 33330
True
here
33299 33463
True
here
33536 33904
True
here
34087 34287
True
here
35825 36259
True
here
3627

# Make CDS datatables for each of those genomes by merging the genome `fasta` file with the CDS `fasta` file

**et voila**

# Scratch

In [29]:
df['Host_species_name'].value_counts()[:20]

Mycolicibacterium smegmatis    1978
Escherichia coli               1279
Salmonella enterica             423
Pseudomonas aeruginosa          400
Gordonia terrae                 382
Lactococcus lactis              358
Klebsiella pneumoniae           251
Arthrobacter sp. ATCC 21022     231
Staphylococcus aureus           221
Microbacterium foliorum         208
Streptococcus thermophilus      199
Cutibacterium acnes             182
Streptococcus pneumoniae        141
Acinetobacter baumannii         116
Vibrio cholerae                 115
Enterococcus faecalis           110
Bacillus thuringiensis          105
Streptococcus pyogenes          101
Streptomyces griseus             97
Salmonella sp.                   79
Name: Host_species_name, dtype: int64

In [84]:
temp_df.sort_values('Length')

,Accession,Release_Date,Species,Genus,Family,Length,Sequence_Type,Nuc_Completeness,Genotype,Segment,...,Host_phylum_id,Host_phylum_name,Host_class_id,Host_class_name,Host_order_id,Host_order_name,Host_family_id,Host_family_name,Host_species_id,Host_species_name
11809,FJ483838,2009-09-09T00:00:00Z,Escherichia virus BZ13,Levivirus,Leviviridae,3393,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11098,JX625144,2012-11-29T00:00:00Z,Enterobacteria phage M,Levivirus,Leviviridae,3405,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
1711,NC_019707,2012-12-07T00:00:00Z,Enterobacteria phage M,Levivirus,Leviviridae,3405,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11808,FJ483837,2009-09-09T00:00:00Z,Escherichia virus BZ13,Levivirus,Leviviridae,3412,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11810,FJ483839,2009-09-09T00:00:00Z,Escherichia virus BZ13,Levivirus,Leviviridae,3458,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,MK817115,2019-06-29T00:00:00Z,Escherichia phage vB_EcoM_phAPEC6,Asteriusvirus,Myoviridae,352598,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
6766,MH383160,2018-07-21T00:00:00Z,Escherichia phage UB,Asteriusvirus,Myoviridae,353081,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
5544,MH791411,2019-02-06T00:00:00Z,Escherichia phage Ecwhy_1,Asteriusvirus,Myoviridae,354537,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
5428,MK327931,2019-03-25T00:00:00Z,Escherichia phage vB_EcoM_G17,Asteriusvirus,Myoviridae,370817,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli


In [89]:
temp_df[temp_df['Species'].str.contains('phi')]

,Accession,Release_Date,Species,Genus,Family,Length,Sequence_Type,Nuc_Completeness,Genotype,Segment,...,Host_phylum_id,Host_phylum_name,Host_class_id,Host_class_name,Host_order_id,Host_order_name,Host_family_id,Host_family_name,Host_species_id,Host_species_name
912,NC_028656,2015-12-17T00:00:00Z,Enterobacteria phage VT2phi_272,Oslovirus,Podoviridae,65955,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
1075,NC_021190,2015-05-28T00:00:00Z,Enterobacteria phage phi80,Ravinvirus,Siphoviridae,46150,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
1120,NC_023693,2014-11-25T00:00:00Z,Enterobacteria phage phi92,NaN,Myoviridae,148612,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
2147,NC_010324,2008-02-11T00:00:00Z,Escherichia virus phiEco32,Kuravirus,Podoviridae,77554,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
2154,NC_010106,2007-12-04T00:00:00Z,Enterobacteria phage phiEcoM-GJ1,NaN,Myoviridae,52975,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11987,EF380030,2007-02-19T00:00:00Z,Escherichia virus phiX174,Sinsheimervirus,Microviridae,5386,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11988,EF380031,2007-02-19T00:00:00Z,Escherichia virus phiX174,Sinsheimervirus,Microviridae,5386,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11989,EF380032,2007-02-19T00:00:00Z,Escherichia virus phiX174,Sinsheimervirus,Microviridae,5386,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
12022,DQ126339,2006-03-01T00:00:00Z,Escherichia virus phiV10,Uetakevirus,Podoviridae,39104,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli


In [90]:
import numpy as np

In [98]:
np.percentile(temp_df['Length'], 75)

88007.0

In [99]:
temp_df.columns

Index(['Accession', 'Release_Date', 'Species', 'Genus', 'Family', 'Length',
       'Sequence_Type', 'Nuc_Completeness', 'Genotype', 'Segment', 'Authors',
       'Publications', 'Geo_Location', 'Host', 'Isolation_Source',
       'Collection_Date', 'BioSample', 'GenBank_Title', 'GeneticCode',
       'Host_genus_id', 'Host_genus_name', 'Host_superkingdom_id',
       'Host_superkingdom_name', 'Host_phylum_id', 'Host_phylum_name',
       'Host_class_id', 'Host_class_name', 'Host_order_id', 'Host_order_name',
       'Host_family_id', 'Host_family_name', 'Host_species_id',
       'Host_species_name'],
      dtype='object')

In [103]:
temp_df['GeneticCode'].value_counts()

11    1279
Name: GeneticCode, dtype: int64

In [110]:
df['GeneticCode'].value_counts()

11    11141
Name: GeneticCode, dtype: int64

In [119]:
df['Family'].value_counts()

Siphoviridae          6079
Myoviridae            1951
Podoviridae            912
Microviridae           479
Autographiviridae      460
Herelleviridae         298
Drexlerviridae         184
Demerecviridae         112
Ackermannviridae        95
Inoviridae              81
Cystoviridae            31
Leviviridae             22
Tectiviridae            12
Sphaerolipoviridae       4
Corticoviridae           2
Plasmaviridae            1
Name: Family, dtype: int64

In [125]:
temp_df = df[df['Host_species_id']==562]



In [128]:
temp_df.sort_values('Length').head(n=30)

,Accession,Release_Date,Species,Genus,Family,Length,Sequence_Type,Nuc_Completeness,Genotype,Segment,...,Host_phylum_id,Host_phylum_name,Host_class_id,Host_class_name,Host_order_id,Host_order_name,Host_family_id,Host_family_name,Host_species_id,Host_species_name
11809,FJ483838,2009-09-09T00:00:00Z,Escherichia virus BZ13,Levivirus,Leviviridae,3393,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11098,JX625144,2012-11-29T00:00:00Z,Enterobacteria phage M,Levivirus,Leviviridae,3405,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
1711,NC_019707,2012-12-07T00:00:00Z,Enterobacteria phage M,Levivirus,Leviviridae,3405,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11808,FJ483837,2009-09-09T00:00:00Z,Escherichia virus BZ13,Levivirus,Leviviridae,3412,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11810,FJ483839,2009-09-09T00:00:00Z,Escherichia virus BZ13,Levivirus,Leviviridae,3458,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
2512,NC_001426,1993-04-28T00:00:00Z,Escherichia virus BZ13,Levivirus,Leviviridae,3466,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11173,JQ966307,2012-09-18T00:00:00Z,Escherichia virus MS2,Levivirus,Leviviridae,3525,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
5791,MK213795,2019-01-08T00:00:00Z,Escherichia virus MS2,Levivirus,Leviviridae,3526,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
2486,NC_001890,1998-04-25T00:00:00Z,Escherichia virus Qbeta,Allolevivirus,Leviviridae,4215,RefSeq,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
11811,FJ483840,2009-09-09T00:00:00Z,Escherichia virus Qbeta,Allolevivirus,Leviviridae,4218,GenBank,complete,NaN,NaN,...,1224.0,Proteobacteria,1236.0,Gammaproteobacteria,91347.0,Enterobacterales,543.0,Enterobacteriaceae,562,Escherichia coli
